In [556]:
import pandas as pd
import numpy as np
import plotly.express as px 


In [557]:
discount_rate = 0.0541 #nominal
inflation = 0.003
monthly_solar_radiation = [5.9*31, 6.4*28, 7*31, 7*30, 6.8*31, 6.9*30, 7.1*31, 7.2*31, 6.7*30, 6.1*31, 5.6*30, 5.3*31]
degradation_rate = 0.00469
electricity_tariff = 0.5444
electricity_escalation_rate = 0.0208
feed_in_tariff = electricity_tariff*0.50
kw10 = 9.787
ic10 = 19667
kw100 = 99.920
ic100 = 180833
kw50 = 49.813
ic50 = 91571


In [558]:
eg = list(i * 10 for i in monthly_solar_radiation)
eg.reverse()
print((eg))

[1642.9999999999998, 1680.0, 1891.0, 2010.0, 2232.0, 2201.0, 2070.0, 2108.0, 2100, 2170, 1792.0000000000002, 1829.0]


In [559]:
def NPV_formula(eer,inf, kw, ic, d, msr, dr, et, fit, eu, delay): #delay=1 if acquiring a permit takes 2 years, 0 if permit is issued immediately 
    NPV =[]
    for l in eu:
        npv = -ic
        for i in range(25):
            yearly_value = 0
            monthly_electricity_generated = list(j * kw * (1-dr*i) for j in msr) 
            for k in monthly_electricity_generated:
                monthly_value = kw*-15*(1+inf)**(i+1)
                if k > l:
                    monthly_value += l * et*(1+eer)**(i+1)
                    if delay != 2 and i > 1:
                        monthly_value += (k-l)*fit*(1+eer)**(i+1)
                else:
                    monthly_value += k*et*(1+eer)**(i+1)
                yearly_value += monthly_value
            npv+=(yearly_value/(1+d)**(i+1))
        NPV.append(round(npv))
    return NPV        

In [560]:
def SNPV_formula(eer, inf, kw, ic, d, msr, dr, et, fit, eu, delay):
    NPV = -ic
    for i in range(25):
        yearly_value = 0
        monthly_electricity_generated = list(j * kw * (1-dr*i) for j in msr) 
        for k in monthly_electricity_generated:
            monthly_value = kw*-15*(1+inf)**(i+1)
            if k > eu:
                monthly_value += eu * et*(1+eer)**(i+1)
                if delay != 2 and i > 1:
                    monthly_value += (k-eu)*fit*(1+eer)**(i+1)
            else:
                monthly_value += k*et*(1+eer)**(i+1)
            yearly_value += monthly_value
        NPV+=(yearly_value/(1+d)**(i+1))
    return NPV        

In [561]:
def SPP_formula(eer, inf, kw, ic, d, msr, dr, et, fit, eu, delay):
    SPP = []
    for l in eu:
        NPV = -ic
        for i in range(25):
            yearly_value = 0
            monthly_electricity_generated = list(j * kw * (1-dr*i) for j in msr) 
            for k in monthly_electricity_generated:
                monthly_value = kw*-15*(1+inf)**(i+1)
                if k > l:
                    monthly_value += l * et*(1+eer)**(i+1)
                    if delay != 2 and i > 1:
                        monthly_value += (k-l)*fit*(1+eer)**(i+1)
                else:
                    monthly_value += k*et*(1+eer)**(i+1)
                yearly_value += monthly_value
            NPV+=(yearly_value/(1+d)**(i+1))
            if NPV > 0:
                spp = i+1
                break
        if NPV > 0:
            monthly_electricity_generated.reverse()
            month_counter = 1
            for k in monthly_electricity_generated:
                month_counter -= 0.083
                monthly_value = kw*-15*(1+inf)**(i+1)
                if k > l:
                    monthly_value += l * et*(1+eer)**(i+1)
                    if delay != 2 and i > 1:
                        monthly_value += (k-l)*fit*(1+eer)**(i+1)
                else:
                    monthly_value += k*et*(1+eer)**(i+1)
                NPV -= monthly_value/(1+d)**(i+1)
                if NPV < 0:
                    spp += round(month_counter,2)
                    monthly_electricity_generated.reverse()
                    break
            SPP.append(spp)
        else:
            SPP.append(0)
    return SPP

In [562]:
def IRR_formula(eer, inf, kw, ic, msr, dr, et, fit, eu, delay):
    IRR = []
    for i in eu:
        lower_bound = -0.5
        upper_bound = 1
        tolerance = 1e-4
        max_iterations = 100
        iteration = 0

        while iteration < max_iterations:
            mid_rate = (lower_bound + upper_bound) / 2
            if mid_rate == 0:
                mid_rate += 1e-4
            npv = SNPV_formula(eer, inf, kw, ic, mid_rate, msr, dr, et, fit, i, delay)
            if abs(npv) < tolerance: 
                IRR.append(round(mid_rate*100, 3))
                break
            elif npv > 0:
                lower_bound = mid_rate  
            else:
                upper_bound = mid_rate 
            iteration += 1
        if iteration >= max_iterations:
            IRR.append(-100)  #IRR is too small to calculate
    return IRR

In [563]:
def LCOE_formula(inf, kw, ic, d, msr, dr, et, fit, eu, fees):
    NPV_costs = ic
    NPV_energy = 0
    for i in range(25):
        if fees ==1:
            NPV_costs += 12*kw*15*(1+inf)**(1+i)/(1+d)**(i+1)
        NPV_energy += (sum(msr) * kw * (1 - dr*i)) / (1+d)**(i+1)
    return NPV_costs/NPV_energy
    

In [564]:
def ROI_formula (eer, inf, kw, ic, d, msr, dr, et, fit, eu, delay):
    ROI = []
    for j in eu:
        NPV_return = SNPV_formula(eer, inf, kw, ic, d, msr, dr, et, fit, j, delay)
        NPV_costs = ic
        for i in range(25):
            NPV_costs += 12*kw*15*(1+inf)**(1+i)/(1+d)**(i+1)
        ROI.append((NPV_return/NPV_costs)*100)
    return ROI

In [565]:
def ROI_graph(x_range):
    x = np.array(x_range)
    y10 = (ROI_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y10d = (ROI_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (ROI_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    y100d = (ROI_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})

    y50 = (ROI_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})

    y50d = (ROI_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})

    fig = px.line(df10, x='x', y='y', title=f'ROI of a different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'ROI (%)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='cyan', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='darkblue', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='cyan', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='darkblue', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='cyan', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='darkblue',dash='dot'))

    fig.update_layout(
    xaxis_title='<b>Monthly Electricity Consumption (kWh)</b>',
    xaxis_title_font_size=9,
    xaxis_title_font_family='Arial',
    xaxis_tickfont_size=12,
    xaxis_tickfont_family='Arial',
    
    yaxis_title='<b>Return on Investment in %</b>',
    yaxis_title_font_size=9,
    yaxis_title_font_family='Arial',
    yaxis_tickfont_size=12,
    yaxis_tickfont_family='Arial',
    
    legend_title_text='', 
    font_family='Arial',
    font_size=6,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin_l=50,
    margin_r=50,
    margin_t=80,
    margin_b=50,

    legend= dict(
        bgcolor='rgba(255, 255, 255, 0.8)',  
        bordercolor='black', 
        borderwidth=1, 
        font_size=9
    ),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
    )
    fig.show()

In [566]:
def IRR_graph(x_range):
    x = np.array(x_range)
    y10 = (IRR_formula(electricity_escalation_rate, inflation, kw10, ic10, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})
    y10d = (IRR_formula(electricity_escalation_rate, inflation, kw10, ic10, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (IRR_formula(electricity_escalation_rate, inflation, kw100, ic100, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})
    y100d = (IRR_formula(electricity_escalation_rate, inflation, kw100, ic100, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})

    y50 = (IRR_formula(electricity_escalation_rate, inflation, kw50, ic50, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})
    y50d = (IRR_formula(electricity_escalation_rate, inflation, kw50, ic50, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})

    fig = px.line(df10, x='x', y='y', title=f'IRR of different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'IRR in %'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='cyan', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='darkblue', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='cyan', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='darkblue', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='cyan', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='darkblue', dash='dot'))    

    fig.update_yaxes(range=[-25, 75])
    
    fig.update_layout(
    xaxis_title='<b>Monthly Electricity Consumption (kWh)</b>',
    xaxis_title_font_size=9,
    xaxis_title_font_family='Arial',
    xaxis_tickfont_size=12,
    xaxis_tickfont_family='Arial',
    
    yaxis_title='<b>Internal Rate of Return in %</b>',
    yaxis_title_font_size=9,
    yaxis_title_font_family='Arial',
    yaxis_tickfont_size=12,
    yaxis_tickfont_family='Arial',
    
    legend_title_text='', 
    font_family='Arial',
    font_size=6,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin_l=50,
    margin_r=50,
    margin_t=80,
    margin_b=50,

    legend= dict(
        bgcolor='rgba(255, 255, 255, 0.8)',  
        bordercolor='black', 
        borderwidth=1, 
        font_size=9
    ),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
    )
    fig.show()

In [567]:
def SPP_graph(x_range):
    x = np.array(x_range)
    y10 = (SPP_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})
    y10d = (SPP_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (SPP_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})
    y100d = (SPP_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})
    df100d = df100d[df100d['y'] > 0]

    y50 = (SPP_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})
    y50d = (SPP_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})
    df50d = df50d[df50d['y'] > 0]

    fig = px.line(df10, x='x', y='y', title=f'SPP of different systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'Simple Payback Period (years)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='cyan', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='darkblue', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='cyan', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='darkblue', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='cyan', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='darkblue', dash='dot'))

    fig.update_yaxes(range=[1, 30])

    fig.update_layout(
    xaxis_title='<b>Monthly Electricity Consumption (kWh)</b>',
    xaxis_title_font_size=9,
    xaxis_title_font_family='Arial',
    xaxis_tickfont_size=12,
    xaxis_tickfont_family='Arial',
    
    yaxis_title='<b>Simple Payback Period (years)</b>',
    yaxis_title_font_size=9,
    yaxis_title_font_family='Arial',
    yaxis_tickfont_size=12,
    yaxis_tickfont_family='Arial',
    
    legend_title_text='', 
    font_family='Arial',
    font_size=6,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin_l=50,
    margin_r=50,
    margin_t=80,
    margin_b=50,

    legend= dict(
        bgcolor='rgba(255, 255, 255, 0.8)',  
        bordercolor='black', 
        borderwidth=1, 
        font_size=9
    ),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
    )
    fig.show()

In [581]:
def LCOE_graph(x_range):
    x = np.array(x_range)
    y10 = (LCOE_formula(inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 1))
    df10 = pd.DataFrame({'x': x, 'y': y10})
    y10p = (LCOE_formula(inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10p = pd.DataFrame({'x': x, 'y': y10p})

    y100 = (LCOE_formula(inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 1))
    df100 = pd.DataFrame({'x': x, 'y': y100})
    y100p = (LCOE_formula(inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100p = pd.DataFrame({'x': x, 'y': y100p})

    y50 = (LCOE_formula(inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 1))
    df50 = pd.DataFrame({'x': x, 'y': y50})
    y50p = (LCOE_formula(inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50p = pd.DataFrame({'x': x, 'y': y50p})

    fig = px.line(df10, x='x', y='y', title=f'LCOE of different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'LCOE (kWh/AWG)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='darkblue', dash='solid'))
    #fig.add_scatter(x=df10p['x'], y=df10p['y'], mode='lines', name='10 kWp System without Fees', line=dict(color='cyan', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='darkblue', dash='dash'))
    #fig.add_scatter(x=df50p['x'], y=df50p['y'], mode='lines', name='50 kWp System without Fees', line=dict(color='cyan', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='darkblue', dash='dot'))
    #fig.add_scatter(x=df100p['x'], y=df100p['y'], mode='lines', name='100 kWp System without Fees', line=dict(color='cyan', dash='dot'))


    fig.add_trace(go.Scatter(
        x=[min(x), max(x)], 
        y=[electricity_tariff, electricity_tariff],        
        mode='lines',
        name='Cost of electricity from the grid',  
        line=dict(color='cyan', width=2)
        ))

    fig.update_layout(
    xaxis_title='<b>Monthly Electricity Consumption (kWh)</b>',
    xaxis_title_font_size=9,
    xaxis_title_font_family='Arial',
    xaxis_tickfont_size=12,
    xaxis_tickfont_family='Arial',
    
    yaxis_title='<b>LCOE (AWG/kWh)</b>',
    yaxis_title_font_size=9,
    yaxis_title_font_family='Arial',
    yaxis_tickfont_size=12,
    yaxis_tickfont_family='Arial',
    
    legend_title_text='', 
    font_family='Arial',
    font_size=6,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin_l=50,
    margin_r=50,
    margin_t=80,
    margin_b=50,

    legend= dict(
        bgcolor='rgba(255, 255, 255, 0.8)',  
        bordercolor='black', 
        borderwidth=1, 
        font_size=9
    ),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
    )
    fig.show()

In [569]:
def NPV_graph(x_range):
    x = np.array(x_range)
    y10 = (NPV_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y10d = (NPV_formula(electricity_escalation_rate, inflation, kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df10d = pd.DataFrame({'x': x, 'y': y10d})

    y100 = (NPV_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    y100d = (NPV_formula(electricity_escalation_rate, inflation, kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df100d = pd.DataFrame({'x': x, 'y': y100d})

    y50 = (NPV_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 0))
    df50 = pd.DataFrame({'x': x, 'y': y50})

    y50d = (NPV_formula(electricity_escalation_rate, inflation, kw50, ic50, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x, 2))
    df50d = pd.DataFrame({'x': x, 'y': y50d})

    fig = px.line(df10, x='x', y='y', title=f'NPV of a different kWp systems depending on electricity consumption', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'NPV (AWG)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(color='cyan', dash='solid'))
    fig.add_scatter(x=df10d['x'], y=df10d['y'], mode='lines', name='10 kWp System with Delay', line=dict(color='darkblue', dash='solid'))

    fig.add_scatter(x=df50['x'], y=df50['y'], mode='lines', name='50 kWp System', line=dict(color='cyan', dash='dash'))
    fig.add_scatter(x=df50d['x'], y=df50d['y'], mode='lines', name='50 kWp System with Delay', line=dict(color='darkblue', dash='dash'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System', line=dict(color='cyan', dash='dot'))
    fig.add_scatter(x=df100d['x'], y=df100d['y'], mode='lines', name='100 kWp System with Delay', line=dict(color='darkblue',dash='dot'))

    fig.update_layout(
    xaxis_title='<b>Monthly Electricity Consumption (kWh)</b>',
    xaxis_title_font_size=9,
    xaxis_title_font_family='Arial',
    xaxis_tickfont_size=12,
    xaxis_tickfont_family='Arial',
    
    yaxis_title='<b>NPV (AWG)</b>',
    yaxis_title_font_size=9,
    yaxis_title_font_family='Arial',
    yaxis_tickfont_size=12,
    yaxis_tickfont_family='Arial',
    
    legend_title_text='', 
    font_family='Arial',
    font_size=6,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin_l=50,
    margin_r=50,
    margin_t=80,
    margin_b=50,

    legend= dict(
        bgcolor='rgba(255, 255, 255, 0.8)',  
        bordercolor='black', 
        borderwidth=1, 
        font_size=9
    ),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
    )
    fig.show()

In [570]:
ROI_graph(x_range=np.linspace(1000, 30000, 100))

In [571]:
IRR_graph(x_range=np.linspace(1000, 30000, 100))

In [572]:
SPP_graph(x_range=np.linspace(1000, 30000, 1000))

In [582]:
LCOE_graph(x_range=np.linspace(1000, 30000, 1000))

In [574]:
NPV_graph(x_range=np.linspace(1000, 30000, 1000))
          

In [575]:
def optimal_calculator(eu, kw, ic):
    optimal = []
    for i in eu:
        for j in range(len(kw)):
            highest_npv = 0
            npv_current  = SNPV_formula(electricity_escalation_rate, inflation, kw[j], ic[j], discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, i, 2)    
            if highest_npv < npv_current:
                highest_npv = npv_current
                optimal_kw = kw[j]
        optimal.append(optimal_kw)
    return optimal
         

In [576]:
def ic_calculator(kw):
    return kw*1788.0+2166.95

monthly_electricity_consumption = np.linspace(1000, 100000, 100)
kw_options = np.linspace(10, 1100, 220)
according_initial_cost = []
for kw in kw_options:
    according_initial_cost.append(ic_calculator(kw))


In [583]:
def optimal_graph(eu, kw, ic):
    x = eu
    y = optimal_calculator(eu, kw, ic)
    df = pd.DataFrame({'x': x, 'y': y})

    fig = px.line(df, x='x', y='y', title='Optimal amount of kWp for maximum NPV considering permit delay', labels={'x': 'Monthly Electricity Consumption (kWh)', 'y': 'Amount of kWp installed'})

    fig.add_scatter(x=df['x'][::5], y=df['y'][::5], mode='lines+markers', name='Most valuable kWp installation given electricity usage', line=dict(color='cyan', dash='solid', width=2), marker=dict(symbol='square', size=8))

    fig.add_trace(go.Scatter(
        x=[min(x), max(x)], 
        y=[100, 100],        
        mode='lines',
        name='Maximum amount of kWp allowed by the government',  
        line=dict(color='darkblue', width=2)
        ))
    fig.update_xaxes(range=[0, 40000])
    fig.update_yaxes(range=[0, 1000])

    fig.update_layout(
    title_text='<b>Figure 2:</b> <i>Optimal amount of kWp for maximum NPV considering permit delay</i>',
    title_font_size=18,
    title_x=0, 
    title_font_family='Times New Roman',
    
    xaxis_title='<b>Monthly Electricity Consumption (kWh)</b>',
    xaxis_title_font_size=9,
    xaxis_title_font_family='Arial',
    xaxis_tickfont_size=12,
    xaxis_tickfont_family='Arial',
    
    yaxis_title='<b>Amount of kWp installed</b>',
    yaxis_title_font_size=9,
    yaxis_title_font_family='Arial',
    yaxis_tickfont_size=12,
    yaxis_tickfont_family='Arial',
    
    legend_title_text='', 
    font_family='Arial',
    font_size=6,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin_l=50,
    margin_r=50,
    margin_t=80,
    margin_b=50,

    legend= dict(
        bgcolor='rgba(255, 255, 255, 0.8)',  
        bordercolor='black', 
        borderwidth=1, 
        font_size=9
    ),
    shapes=[
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=0, y0=0, x1=1, y1=1,
            line=dict(color="black", width=2)
        )
    ]
    )
    fig.show()

In [584]:
optimal_graph(monthly_electricity_consumption, kw_options, according_initial_cost)